# LEC-3

In [18]:
import torch

In [19]:
# Doing Derivative without using 'argmax'
def dy_dx(x):
  return 2*x  # Equation of dy/dx

dy_dx(3)

6

In [20]:
# Now for derivation chain, where y=x^2 and z=sin(y), getting dz/dx is tough
import math

def dz_dx(x):
  return 2*x*math.cos(x**2) # Equation of dz/dx

dz_dx(3)

-5.466781571308061

# Using Autograd

In [21]:
import torch

## Example 1: y = x^2

In [22]:
# requires_grad -> to get derivatives later
x = torch.tensor(3.0, requires_grad=True)
y = x**2
print(x, y)
y.backward() # dy/dx = 2x
x.grad # x.grad contains dy/dx

tensor(3., requires_grad=True) tensor(9., grad_fn=<PowBackward0>)


tensor(6.)

## Example 2: y = x^2, z = sin(y)

In [24]:
x = torch.tensor(3.0, requires_grad=True)
y = x**2 # y = x^2
z = torch.sin(y) # z = sin(y)
print(x, y, z)
z.backward() # dz/dx
x.grad # same when without using autograd

tensor(3., requires_grad=True) tensor(9., grad_fn=<PowBackward0>) tensor(0.4121, grad_fn=<SinBackward0>)


tensor(-5.4668)